# Protocols - Protocol-Based Composition

Protocols enable **structural typing** (duck typing with explicit declarations) in lionherd-core. This follows **Rust traits** and **Go interfaces** philosophy: composition over inheritance, loose coupling, and explicit capability declarations.

**Key Features:**
- **Structural Typing**: Compatible if methods match, not if inheritance matches
- **Loose Coupling**: No shared base class requirements
- **Explicit Contracts**: `@implements()` decorator declares capabilities
- **Runtime Checking**: `isinstance(obj, Protocol)` validates compatibility

In [1]:
from uuid import UUID, uuid4

from lionherd_core.protocols import (
    Deserializable,
    Hashable,
    Observable,
    Serializable,
    implements,
)

## 1. Observable Protocol - UUID Identity

Objects with unique UUID identifier for tracking and identity-based operations.

In [2]:
# Implement Observable protocol
@implements(Observable)
class Agent:
    def __init__(self, name: str):
        self._id = uuid4()
        self.name = name

    @property
    def id(self) -> UUID:
        return self._id


# Create agent
agent = Agent("agent_1")
print(f"ID: {agent.id}")
print(f"Name: {agent.name}")

# Runtime protocol checking
print(f"Is Observable: {isinstance(agent, Observable)}")

ID: 105b7fd8-75b3-4197-b691-d1f8ca42cdc0
Name: agent_1
Is Observable: True


## 2. Serializable Protocol - Dictionary Serialization

Objects that can be serialized to dictionary representation.

In [3]:
@implements(Serializable)
class Config:
    def __init__(self, host: str, port: int):
        self.host = host
        self.port = port

    def to_dict(self, **kwargs):
        return {"host": self.host, "port": self.port}


# Create and serialize
config = Config("localhost", 8080)
data = config.to_dict()
print(f"Serialized: {data}")

# Runtime check
print(f"Is Serializable: {isinstance(config, Serializable)}")

Serialized: {'host': 'localhost', 'port': 8080}
Is Serializable: True


## 3. Multiple Protocols with @implements()

Classes can implement multiple protocols for rich capabilities.

In [4]:
@implements(Observable, Serializable, Deserializable, Hashable)
class Task:
    def __init__(self, task_id: UUID | None = None, title: str = ""):
        self._id = task_id or uuid4()
        self.title = title

    @property
    def id(self) -> UUID:
        return self._id

    def to_dict(self, **kwargs):
        return {"id": str(self.id), "title": self.title}

    @classmethod
    def from_dict(cls, data: dict, **kwargs):
        return cls(task_id=UUID(data["id"]), title=data["title"])

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return isinstance(other, Task) and self.id == other.id


# Create task
task = Task(title="Build feature")

# Observable
print(f"Task ID: {task.id}")

# Serializable
data = task.to_dict()
print(f"Serialized: {data}")

# Deserializable
restored = Task.from_dict(data)
print(f"Restored: {restored.title}")

# Hashable - use in sets/dicts
tasks = {task, Task(title="Other task")}
print(f"Set size: {len(tasks)}")
lookup = {task: "in_progress"}
print(f"Dict lookup: {lookup[task]}")

Task ID: 401c4a1b-26a9-4d00-95ab-c3a14b855f48
Serialized: {'id': '401c4a1b-26a9-4d00-95ab-c3a14b855f48', 'title': 'Build feature'}
Restored: Build feature
Set size: 2
Dict lookup: in_progress


## 4. Why NOT Inherit from Protocols (Anti-Pattern)

Protocols are NOT base classes. Use @implements() instead.

In [5]:
# ❌ WRONG: Inheriting from protocol
# class BadClass(Observable):  # Don't do this!
#     pass

# ✅ CORRECT: Implement protocol methods with @implements()
@implements(Observable)
class GoodClass:
    def __init__(self):
        self._id = uuid4()

    @property
    def id(self):
        return self._id


good = GoodClass()
print(f"Is Observable: {isinstance(good, Observable)}")
print(f"Has __protocols__: {hasattr(GoodClass, '__protocols__')}")
print(f"Declared protocols: {GoodClass.__protocols__}")

Is Observable: True
Has __protocols__: True
Declared protocols: (<class 'lionherd_core.protocols.ObservableProto'>,)


## 5. @implements() Requires Literal Implementation

CRITICAL: Methods must be defined in class body, NOT inherited from parent.

In [6]:
class Parent:
    def to_dict(self, **kwargs):
        return {"parent": "data"}


# ❌ WRONG: Relying on inherited method
# @implements(Serializable)  # Violation!
# class WrongChild(Parent):
#     pass  # to_dict inherited, not implemented in body


# ✅ CORRECT: Explicit override in class body
@implements(Serializable)
class CorrectChild(Parent):
    def to_dict(self, **kwargs):  # Explicit in body
        data = super().to_dict(**kwargs)
        data["child"] = "additional"
        return data


child = CorrectChild()
print(f"Serialized: {child.to_dict()}")
print(f"Is Serializable: {isinstance(child, Serializable)}")

Serialized: {'parent': 'data', 'child': 'additional'}
Is Serializable: True


## 6. Runtime Protocol Checking

Use `isinstance()` to validate objects implement required protocols.

In [7]:
def process_observable(obj):
    """Process object that MUST be Observable."""
    if not isinstance(obj, Observable):
        raise TypeError(f"{type(obj).__name__} must implement Observable")
    return obj.id


# Valid object
agent = Agent("agent_1")
agent_id = process_observable(agent)
print(f"Processed agent ID: {agent_id}")

# Invalid object
try:
    process_observable("string")
except TypeError as e:
    print(f"✓ TypeError caught: {e}")

Processed agent ID: e9803a0a-a1d7-4e51-a919-2d037f17ac59
✓ TypeError caught: str must implement Observable


## 7. Protocol Composition - Rich Capabilities

Combine protocols for objects with multiple capabilities.

In [8]:
@implements(Observable, Serializable, Hashable)
class Entity:
    def __init__(self, name: str):
        self._id = uuid4()
        self.name = name

    @property
    def id(self) -> UUID:
        return self._id

    def to_dict(self, **kwargs):
        return {"id": str(self.id), "name": self.name}

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return isinstance(other, Entity) and self.id == other.id


# Create entities
e1 = Entity("entity_1")
e2 = Entity("entity_2")
e3 = Entity("entity_3")

# Observable - track by ID
print(f"Entities: {[e.id for e in [e1, e2, e3]]}")

# Serializable - convert to dict
serialized = [e.to_dict() for e in [e1, e2, e3]]
print(f"Serialized count: {len(serialized)}")

# Hashable - use in sets/dicts
entity_set = {e1, e2, e3, e1}  # e1 deduplicated
print(f"Set size (e1 deduplicated): {len(entity_set)}")

lookup = {e1: "active", e2: "pending", e3: "completed"}
print(f"Lookup[e2]: {lookup[e2]}")

Entities: [UUID('5af9dba5-f51d-4d71-ac96-1e374e4d10ec'), UUID('d2a6281c-2968-472b-a654-901e02fd64b1'), UUID('6fd20270-76b8-46b8-ba9d-e1ac46a2494b')]
Serialized count: 3
Set size (e1 deduplicated): 3
Lookup[e2]: pending


## 8. Polymorphic Collections

Protocols enable type-safe polymorphism without shared base class.

In [9]:
def serialize_all(items) -> list[dict]:
    """Serialize heterogeneous collection (all must be Serializable)."""
    results = []
    for item in items:
        if not isinstance(item, Serializable):
            raise TypeError(f"{type(item).__name__} must implement Serializable")
        results.append(item.to_dict())
    return results


# Mixed collection (different classes, same protocol)
items = [
    Config("localhost", 8080),
    Task(title="Build feature"),
    Entity("entity_1"),
]

# Serialize all
serialized = serialize_all(items)
print(f"Serialized {len(serialized)} items:")
for i, data in enumerate(serialized):
    print(f"  {i + 1}. {data}")

Serialized 3 items:
  1. {'host': 'localhost', 'port': 8080}
  2. {'id': '05afaeeb-6981-42da-aab3-b1bec8ac4ffe', 'title': 'Build feature'}
  3. {'id': 'f875fffc-9b91-41a7-9622-efe9fad591a0', 'name': 'entity_1'}


## 9. Common Pitfalls Demonstration

In [10]:
# Pitfall 1: @implements() strict enforcement (NEW in v1.0.0-alpha4)
# BEFORE PR #149: Would create class, fail at runtime when accessing .id
# AFTER PR #149: Fails at decoration time with clear error message

try:

    @implements(Observable)
    class IncompleteClass:
        pass  # Missing .id property
except TypeError as e:
    print(f"✓ Strict enforcement: {e}")
    print("\nThis is GOOD! Catches the error immediately at decoration time.")


# Pitfall 2: Hash/Equality inconsistency
@implements(Hashable)
class BadHash:
    def __init__(self, value):
        self.value = value

    def __hash__(self):
        return hash(self.value)

    # Missing __eq__! Default is identity-based


h1 = BadHash(42)
h2 = BadHash(42)
print(f"\nSame hash: {hash(h1) == hash(h2)}")
print(f"Equal: {h1 == h2}")  # False! Different objects
print("⚠️  Hash collision without equality - bad for dicts/sets")

✓ Strict enforcement: IncompleteClass declares @implements(ObservableProto) but does not define 'id' in its class body (inheritance not allowed)

This is GOOD! Catches the error immediately at decoration time.

Same hash: True
Equal: False
⚠️  Hash collision without equality - bad for dicts/sets


## 10. Summary Checklist

**Protocols in lionherd-core:**
- ✅ Use `@implements()` decorator to declare protocols
- ✅ Define ALL protocol methods in class body (not via inheritance)
- ✅ Override inherited methods explicitly if using @implements()
- ✅ Implement `__hash__()` and `__eq__()` consistently for Hashable
- ✅ Use `isinstance(obj, Protocol)` for runtime validation
- ✅ Combine multiple protocols for rich capabilities
- ❌ DON'T inherit from protocol classes
- ❌ DON'T rely on inherited methods for @implements()
- ❌ DON'T assume static type hints provide runtime safety

**Core Protocols:**
- **Observable**: UUID identity (`id` property)
- **Serializable**: Dictionary serialization (`to_dict()`)
- **Deserializable**: Dictionary deserialization (`from_dict()`)
- **Hashable**: Hash for sets/dicts (`__hash__()`, `__eq__()`)
- **Adaptable**: Format conversion (`adapt_to()`, `register_adapter()`)

**Next Steps:**
- See `Element` for Observable + Serializable + Hashable implementation
- See `Node` for Adaptable integration with pydapter
- See `Pile` for Containable protocol implementation